📂 1. Import Required Libraries

In [ ]:
# 📌 Import necessary libraries
import os
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv3D, Flatten, MaxPooling3D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import Sequence
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report

🎥 2. Convert Videos to .npy Format

In [ ]:
# 📌 Convert videos to numpy format for faster processing
def convert_videos_to_npy(input_path, output_path):
    class_counters = {"Fight": 1, "NonFight": 1}
    
    for class_name in os.listdir(input_path):
        class_dir = os.path.join(input_path, class_name)
        class_output_dir = os.path.join(output_path, class_name)
        os.makedirs(class_output_dir, exist_ok=True)

        if os.path.isdir(class_dir):
            for video_name in os.listdir(class_dir):
                video_path = os.path.join(class_dir, video_name)
                if video_name.endswith(('.mp4', '.avi', '.mkv')):  
                    cap = cv2.VideoCapture(video_path)
                    frames = []

                    while True:
                        ret, frame = cap.read()
                        if not ret:
                            break
                        frame = cv2.resize(frame, (150, 150))  
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  
                        frames.append(frame)

                    cap.release()
                    frames_array = np.array(frames, dtype=np.uint8)

                    file_prefix = "F" if class_name == "Fight" else "NF"
                    output_file = os.path.join(class_output_dir, f"{file_prefix}_{class_counters[class_name]}.npy")
                    np.save(output_file, frames_array)
                    print(f"Converted and saved: {output_file}")

                    class_counters[class_name] += 1

📦 3. Define the Data Generator

In [ ]:
# 📌 Data Generator to Load Preprocessed `.npy` Files
class DataGenerator(Sequence):
    def __init__(self, data_folder, batch_size=32, dim=(30, 150, 150), n_channels=3, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.data_folder = data_folder
        self.files = []
        self.labels = []

        for label in ['Fight', 'NonFight']:
            folder = os.path.join(data_folder, label)
            for file in os.listdir(folder):
                if file.endswith('.npy'):
                    self.files.append(os.path.join(folder, file))
                    self.labels.append(1 if label == "Fight" else 0)

        self.indexes = np.arange(len(self.files))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.files) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.files))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty(self.batch_size, dtype=int)

        for i, idx in enumerate(indexes):
            X[i,] = np.load(self.files[idx]) / 255.0
            y[i] = self.labels[idx]

        return X, y

📌 4. Define the 3D CNN Model

In [ ]:
# 📌 Define the 3D CNN Model Architecture
def create_model(input_shape):
    model = Sequential()
    model.add(Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(1, 2, 2)))
    
    model.add(Conv3D(64, (3, 3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(1, 2, 2)))
    
    model.add(Conv3D(128, (3, 3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(1, 2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

🚀 5. Train the Model

In [ ]:
# 📌 Train the Model
dataset_folder = "Dataset"
output_folder = "Numpy_Videos"

CONVERT_VIDEOS = True  # Set to True if you need to convert videos

if CONVERT_VIDEOS:
    convert_videos_to_npy(dataset_folder, output_folder)

print("Creating data generators...")
train_generator = DataGenerator(output_folder, batch_size=16)
validation_generator = DataGenerator(output_folder, batch_size=16, shuffle=False)

print("Creating the model...")
input_shape = (30, 150, 150, 3)
model = create_model(input_shape)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

print("Starting model training...")
history = model.fit(train_generator, 
                    validation_data=validation_generator,
                    epochs=20,
                    callbacks=[early_stopping, lr_scheduler])
print("Model training process completed.")

💾 6. Save the Violence Model

In [ ]:
model.save('violence_detection_model.h5')
print("Model saved.")

📊 7. Evaluate the Model

In [ ]:
# 📌 Evaluate Model Performance
print("Evaluating the model...")
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# 📌 Generate Classification Report
y_pred = model.predict(validation_generator)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()
y_true = np.concatenate([label for _, label in validation_generator])
y_true = y_true[:len(y_pred_classes)]

print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes))

📈 8. Plot Confusion Matrix

In [ ]:
# 📌 Plot Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['NonFight', 'Fight'], yticklabels=['NonFight', 'Fight'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()